In [ ]:
fileName = 'honey.mp4'

In [ ]:
import requests

resp = requests.post(
    'https://openapi.vito.ai/v1/authenticate',
    data={'client_id': '본인의_아이디',
          'client_secret': '본인의_시크릿'}
)
resp.raise_for_status()
# print(resp.json())

token = (resp.json()['access_token'])
print(token)

In [ ]:
import json
import requests

config = {
  "diarization": {
    "use_verification": False
  },
  "use_multi_channel": False
}
resp = requests.post(
    'https://openapi.vito.ai/v1/transcribe',
    headers={'Authorization': 'bearer '+token},
    data={'config': json.dumps(config)},
    files={'file': open(fileName, 'rb')}
)
resp.raise_for_status()
# print(resp.json())

ReqId = resp.json()['id']
ReqId

In [ ]:
import requests
import time


while 'results' not in resp.json() :    

    resp = requests.get(
        'https://openapi.vito.ai/v1/transcribe/'+ReqId,
        headers={'Authorization': 'bearer '+token},
    )
    resp.raise_for_status()

    time.sleep(3)

scripts = resp.json()['results']['utterances']
print(f"총 {len(scripts)} 개의 발화가 인식되었습니다")

In [ ]:
def timeConvert(seconds) :
    ms = str(seconds)[-3:]
    seconds = int(str(seconds)[:-3])
    
    if seconds > 3600 :
        hours = str(seconds // 3600)
        seconds = seconds % 3600
    else :
        hours = "00"

    if seconds > 60 :
        mins = str(seconds // 60)
        seconds = seconds % 60
    else :
        mins = "00"
    
    seconds = str(seconds)

    return hours+":"+mins+":"+seconds+","+ms

In [ ]:
# SRT 포멧으로 변경

srtName = fileName.replace('.mp4', '.srt')
with open(srtName, 'w', encoding="utf-8") as f :
    for i in range(len(scripts)) :
        # 자막 index
        f.write(str(i+1))
        f.write("\n")

        # 자막 시간
        start = timeConvert(scripts[i]['start_at'])
        end = timeConvert(scripts[i]['start_at'] + scripts[i]['duration']+100)
        
        f.write(start+" --> "+end)
        f.write("\n")

        # 자막 내용
        f.write(scripts[i]['msg'])
        f.write("\n")
        f.write("\n")

    f.close()